In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://security.ubuntu.c

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-20 15:11:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.56MB/s    in 0.2s    

2022-05-20 15:11:36 (5.56 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [ ]:
# Total_votes Count is equal to or greater than 20 

from pyspark.sql.functions import count
vine_filtered_df= vine_df.filter("total_votes >= 20")
vine_filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R2O4P39D0ALCQK|          2|           14|         43|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|     

In [ ]:
#The number of helpful_votes divided by total_votes is equal to or greater than 50%.

vine_filtered_percent_df = vine_filtered_df.withColumn('50above', vine_filtered_df['helpful_votes'] / vine_filtered_df['total_votes']).filter('50above>=0.5')
vine_filtered_percent_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           50above|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|0.8620689655172413|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|               1.0|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|0.9354838709677419|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|0.9629629629629629|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|0.9523809523809523|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|0.8974358974358975|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|               1.0|
|R1E1PQGHWGO3Q1|          1|  

In [ ]:
# Vine program (paid)
from pyspark.sql.functions import col

vine_filtered_paid_df = vine_filtered_percent_df.filter(col("vine")=="Y")
vine_filtered_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           50above|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3KPC0NBUDASX3|          5|           25|         25|   Y|                N|               1.0|
|R119P2A95GGXX4|          5|           26|         28|   Y|                N|0.9285714285714286|
|R1HKIRME8AJ89Z|          5|           79|         82|   Y|                N|0.9634146341463414|
|R3FY3GMBGOBR22|          5|           12|         20|   Y|                N|               0.6|
|R15KH3FBSVYGBU|          5|           32|         37|   Y|                N|0.8648648648648649|
|R22XZEQLVLWW49|          5|           28|         33|   Y|                N|0.8484848484848485|
|R2OIQYVAL9D63R|          4|           59|         65|   Y|                N|0.9076923076923077|
|R1W3KCDHM6BMZT|          4|  

In [ ]:
#  Vine program (unpaid)
vine_filtered_unpaid_df = vine_filtered_percent_df.filter(col("vine")=="N")
vine_filtered_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           50above|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|0.8620689655172413|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|               1.0|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|0.9354838709677419|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|0.9629629629629629|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|0.9523809523809523|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|0.8974358974358975|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|               1.0|
|R1E1PQGHWGO3Q1|          1|  

In [ ]:
#Determine the total number of reviews paid
from pyspark.sql.functions import sum
total_review_paid = vine_filtered_paid_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_Vine_Review")
total_review_paid.show()

+-----------------+
|Total_Vine_Review|
+-----------------+
|              107|
+-----------------+



In [ ]:
#the number of 5-star reviews paid 
five_star_paid = vine_filtered_paid_df.filter(vine_filtered_paid_df['star_rating' ]== 5).agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5 Star Vine Rating")
five_star_paid.show()

+------------------+
|5 Star Vine Rating|
+------------------+
|                56|
+------------------+



In [ ]:
#Percent of Paid 5-Star Review 
Percent_of_Paid_five_Star_Review  = five_star_paid.collect()[0]['5 Star Vine Rating'] / total_review_paid.collect()[0]['Total_Vine_Review'] * 100
print(f" Percent of Vine 5-Star Review  is :{round(Percent_of_Paid_five_Star_Review,2)}")


 Percent of Vine 5-Star Review  is :52.34


In [ ]:
#Determine the total number of reviews unpaid
from pyspark.sql.functions import sum
total_review_unpaid = vine_filtered_unpaid_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)","Total NonVine Review")
total_review_unpaid.show()


+--------------------+
|Total NonVine Review|
+--------------------+
|               39869|
+--------------------+



In [ ]:
#the number of 5-star reviews unpaid 
five_star_unpaid = vine_filtered_unpaid_df.filter(vine_filtered_unpaid_df['star_rating' ]== 5).agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5 Star Non Vine Rating")
five_star_unpaid.show()

+----------------------+
|5 Star Non Vine Rating|
+----------------------+
|                 21005|
+----------------------+



In [ ]:
#Percent of Paid 5-Star Review 
Percent_of_Unpaid_five_Star_Review  = five_star_unpaid.collect()[0]['5 Star Non Vine Rating'] / total_review_unpaid.collect()[0]['Total NonVine Review'] * 100
print(f" Percent of Unpaid 5-Star Review  is :{round(Percent_of_Unpaid_five_Star_Review ,2)}")


 Percent of Unpaid 5-Star Review  is :52.69
